In [ ]:
%load_ext autoreload
%autoreload 2

from scdesigner.simulators import NegBinRegressionSimulator
import numpy as np
import pandas as pd
import torch
from anndata import AnnData


n, g, d, p = 1000, 20, 2, 2
X1 = np.random.normal(size=(n, d)) # 
X2 = np.random.normal(size=(n, p)) #    
B = np.random.normal(size=(d, g)) # feature x gene
D = np.random.normal(size=(p, g)) # feature x gene
mu = np.exp(X1 @ B) # cell x gene
r = np.exp(X2 @ D) # cell x gene

# generate samples
Y = np.random.negative_binomial(r, r / (r + mu))

X1 = pd.DataFrame(X1, columns=[f"mean_dim{j}" for j in range(d)]) # cell x feature
X2 = pd.DataFrame(X2, columns=[f"dispersion_dim{j}" for j in range(p)]) # cell x feature
obs = pd.concat([X1, X2], axis=1)

adata = AnnData(X=Y, obs=obs)

print("Testing the fixed NegBinRegressionSimulator...")
nb_simulator = NegBinRegressionSimulator()
nb_simulator.fit(adata, {"mean": "~ mean_dim0 + mean_dim1 - 1", 
                                      "dispersion": "~ dispersion_dim0 + dispersion_dim1 - 1"})
print("Success! Parameters fitted successfully.")
print("Parameter keys:", list(nb_simulator.params.keys()))


/opt/miniconda3/envs/scdesign/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Testing the fixed NegBinRegressionSimulator...
{'mean': <torch.utils.data.dataloader.DataLoader object at 0x3144a9df0>, 'dispersion': <torch.utils.data.dataloader.DataLoader object at 0x3150432f0>}
entering multiple formula regression factory


Success! Parameters fitted successfully.
Parameter keys: ['beta_mean', 'beta_dispersion']


In [7]:
nb_simulator.params["beta_mean"]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Intercept,0.301800,1.353521,0.030979,-0.019013,0.032633,0.110472,0.021932,-0.076160,0.020287,-0.017039,-0.050987,0.039863,-0.019633,0.208158,-0.043696,0.156797,-0.025519,0.073174,0.001505,0.018052
mean_dim0,1.541806,-1.284216,0.512772,0.491720,0.840530,0.038129,1.262179,0.894032,0.304259,0.137421,1.406109,-1.251278,-0.004712,-1.191263,1.187740,1.129312,0.206142,0.610185,1.069384,0.591737
mean_dim1,0.920438,0.933623,0.405162,-1.591898,0.279792,-0.925620,-0.811360,-1.190150,0.967301,1.449749,0.985871,0.133237,0.424415,1.278975,0.646334,1.695256,-0.148733,1.184585,0.650981,-0.755815


In [9]:
pd.DataFrame(B)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.889040,-3.200223,0.539616,0.523651,0.849127,-0.018207,1.268293,0.935182,0.239736,0.150316,1.406780,-1.283631,-0.041409,-1.335051,1.136949,1.200183,0.208696,0.687121,1.060379,0.593888
1,1.055448,0.060486,0.284368,-1.597373,0.250950,-1.040168,-0.808817,-1.133786,0.962609,1.448188,0.943965,0.026433,0.424413,1.415980,0.649360,1.864617,-0.112444,1.260022,0.644153,-0.801605
